# 🥈 MoviePulse — Bronze → Silver (Spark + Postgres)

Este notebook lê os dados brutos da camada Bronze (MinIO), aplica transformações e publica no Postgres na camada Silver.

### Modelo Silver
```
silver.dim_genres          → catálogo de gêneros (id + nome)
silver.dim_movies          → 1 linha por filme (deduplicado)
silver.bridge_movie_genres → relação N:N filme ↔ gênero
silver.fact_trending       → 1 linha por filme por dia (métricas)
```

> **Pré-requisito:** rode o notebook `spark_minio_bronze.ipynb` primeiro para garantir que a SparkSession e os JARs estão prontos.

## ⚙️ Passo 1 — SparkSession com S3A + JDBC (Postgres)

In [4]:
import os, subprocess
from pyspark.sql import SparkSession

jar_dir = "/home/jovyan/jars"
os.makedirs(jar_dir, exist_ok=True)

# JARs necessários
JARS_URLS = {
    "hadoop-aws-3.3.4.jar":              "https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar",
    "aws-java-sdk-bundle-1.12.262.jar":  "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar",
    "postgresql-42.7.3.jar":             "https://repo1.maven.org/maven2/org/postgresql/postgresql/42.7.3/postgresql-42.7.3.jar",
}

for filename, url in JARS_URLS.items():
    dest = f"{jar_dir}/{filename}"
    if not os.path.exists(dest):
        print(f"  Baixando {filename}...")
        result = subprocess.run(["wget", "-q", "-O", dest, url], capture_output=True)
        status = "" if result.returncode == 0 else ""
        print(f"{status} {filename}")
    else:
        print(f" {filename} já existe.")

jars = ",".join([f"{jar_dir}/{f}" for f in JARS_URLS.keys()])

spark = (
    SparkSession.builder
    .appName("MoviePulse-Bronze-to-Silver")
    .config("spark.jars", jars)

    # S3A → MinIO
    .config("spark.hadoop.fs.s3a.impl",                    "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.access.key",              "minio")
    .config("spark.hadoop.fs.s3a.secret.key",              "minio123")
    .config("spark.hadoop.fs.s3a.endpoint",                "http://minio:9000")
    .config("spark.hadoop.fs.s3a.path.style.access",       "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled",  "false")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print(f"\n SparkSession pronta! Versão: {spark.version}")

  Baixando hadoop-aws-3.3.4.jar...
 hadoop-aws-3.3.4.jar
  Baixando aws-java-sdk-bundle-1.12.262.jar...
 aws-java-sdk-bundle-1.12.262.jar
  Baixando postgresql-42.7.3.jar...
 postgresql-42.7.3.jar

 SparkSession pronta! Versão: 3.5.0


## 📖 Passo 2 — Ler o Bronze

In [5]:
df_bronze = (
    spark.read
    .option("recursiveFileLookup", "true")
    .option("multiLine", "false")
    .json("s3a://bronze/tmdb/trending_daily/")
)

print(f" Registros no Bronze: {df_bronze.count()}")
df_bronze.printSchema()

 Registros no Bronze: 121
root
 |-- _kafka: struct (nullable = true)
 |    |-- key: string (nullable = true)
 |    |-- offset: long (nullable = true)
 |    |-- partition: long (nullable = true)
 |    |-- topic: string (nullable = true)
 |    |-- ts_ms: long (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- event_type: string (nullable = true)
 |-- extraction_ts: string (nullable = true)
 |-- movie_id: long (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- raw: struct (nullable = true)
 |    |-- adult: boolean (nullable = true)
 |    |-- backdrop_path: string (nullable = true)
 |    |-- genre_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: long (nullable = true)
 |    |-- media_type: string (nullable = true)
 |    |-- original_language: string (nullable = true)
 |    |-- original_title: string (nullable = true)
 |    |-- overview: string (nullable = true)
 |    |-- po

## 🔧 Passo 3 — Limpeza e tipagem dos campos

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DoubleType, DateType, BooleanType, TimestampType

df_clean = (
    df_bronze

    # ── Campos do evento ──────────────────────────────────────────
    .withColumn("movie_id",         F.col("movie_id").cast(IntegerType()))
    .withColumn("title",            F.trim(F.col("title")))
    .withColumn("popularity",       F.col("popularity").cast(DoubleType()))
    .withColumn("vote_average",     F.col("vote_average").cast(DoubleType()))
    .withColumn("vote_count",       F.col("vote_count").cast(IntegerType()))
    .withColumn("adult",            F.col("adult").cast(BooleanType()))
    .withColumn("release_date",     F.to_date(F.col("release_date"), "yyyy-MM-dd"))
    .withColumn("source_date",      F.to_date(F.col("source_date"),  "yyyy-MM-dd"))
    .withColumn("extraction_ts",    F.to_timestamp(F.col("extraction_ts")))
    .withColumn("original_language",F.trim(F.col("original_language")))

    # ── Campos extras do raw (enriquecimento) ────────────────────
    .withColumn("original_title",   F.trim(F.col("raw.original_title")))
    .withColumn("overview",         F.trim(F.col("raw.overview")))
    .withColumn("poster_path",      F.col("raw.poster_path"))
    .withColumn("genre_ids",        F.col("raw.genre_ids"))   # array de inteiros

    # ── Remover linhas sem movie_id (dados corrompidos) ──────────
    .filter(F.col("movie_id").isNotNull())
    .filter(F.col("title").isNotNull())

    # ── Metadado de processamento ────────────────────────────────
    .withColumn("processed_at", F.current_timestamp())
)

print(f" Registros após limpeza: {df_clean.count()}")
df_clean.select("movie_id", "title", "popularity", "vote_average", "release_date", "source_date", "genre_ids").show(5, truncate=False)

 Registros após limpeza: 120
+--------+-------------------------------------+----------+------------+------------+-----------+-------------------------+
|movie_id|title                                |popularity|vote_average|release_date|source_date|genre_ids                |
+--------+-------------------------------------+----------+------------+------------+-----------+-------------------------+
|812583  |Vivo ou Morto: Um Mistério Knives Out|368.6188  |7.408       |2025-11-26  |2025-12-15 |[53, 9648, 18]           |
|812583  |Vivo ou Morto: Um Mistério Knives Out|368.6188  |7.408       |2025-11-26  |2025-12-15 |[53, 9648, 18]           |
|798645  |O Sobrevivente                       |429.9601  |6.776       |2025-11-11  |2025-12-15 |[28, 53, 878]            |
|798645  |O Sobrevivente                       |429.9601  |6.776       |2025-11-11  |2025-12-15 |[28, 53, 878]            |
|1084242 |Zootopia 2                           |431.2602  |7.669       |2025-11-26  |2025-12-15 |[16, 3

## 🎭 Passo 4 — Tabela de gêneros (dim_genres)
O TMDb usa IDs numéricos para gêneros. Buscamos o catálogo oficial via API para resolver os nomes.

In [ ]:
import requests, os

TMDB_API_KEY = os.getenv("TMDB_API_KEY", "")  # vem do docker-compose

# Busca catálogo de gêneros do TMDb
resp = requests.get(
    "https://api.themoviedb.org/3/genre/movie/list",
    params={"api_key": TMDB_API_KEY, "language": "pt-BR"},
    timeout=15
)
resp.raise_for_status()
genres_data = resp.json()["genres"]  # [{"id": 28, "name": "Ação"}, ...]

print(f" {len(genres_data)} gêneros carregados do TMDb:")
for g in genres_data:
    print(f"  {g['id']:>5} → {g['name']}")

In [ ]:
# Cria DataFrame de gêneros
df_genres = spark.createDataFrame(
    [(g["id"], g["name"]) for g in genres_data],
    schema=["genre_id", "genre_name"]
)

df_genres.show(truncate=False)

## 🎬 Passo 5 — dim_movies (1 linha por filme, deduplicado)

In [ ]:
from pyspark.sql.window import Window

# Para cada movie_id, mantém o registro com a extração mais recente
# Isso garante que os campos descritivos (título, overview) estejam atualizados
window_latest = Window.partitionBy("movie_id").orderBy(F.col("extraction_ts").desc())

df_dim_movies = (
    df_clean
    .withColumn("rn", F.row_number().over(window_latest))
    .filter(F.col("rn") == 1)                          # 1 linha por filme
    .select(
        "movie_id",
        "title",
        "original_title",
        "original_language",
        "overview",
        "poster_path",
        "release_date",
        "adult",
        "processed_at",
    )
)

print(f" Filmes únicos (dim_movies): {df_dim_movies.count()}")
df_dim_movies.show(5, truncate=False)

## 🔗 Passo 6 — bridge_movie_genres (N:N filme ↔ gênero)

In [ ]:
# Explode: cada linha vira N linhas (1 por gênero)
df_bridge = (
    df_clean
    .select("movie_id", F.explode("genre_ids").alias("genre_id"))
    .filter(F.col("genre_id").isNotNull())
    .distinct()                                        # remove duplicatas (mesmo filme aparece em vários dias)
)

print(f" Relações filme-gênero: {df_bridge.count()}")
df_bridge.show(10)

## 📈 Passo 7 — fact_trending (métricas diárias por filme)

In [ ]:
df_fact_trending = (
    df_clean
    # Deduplicar por (movie_id, source_date) — pode haver reprocessamento
    .withColumn("rn", F.row_number().over(
        Window.partitionBy("movie_id", "source_date")
              .orderBy(F.col("extraction_ts").desc())
    ))
    .filter(F.col("rn") == 1)
    .select(
        "movie_id",
        "source_date",
        "popularity",
        "vote_average",
        "vote_count",
        "extraction_ts",
        "processed_at",
    )
)

print(f" Registros de trending (fact): {df_fact_trending.count()}")
df_fact_trending.orderBy(F.col("popularity").desc()).show(10)

## 🐘 Passo 8 — Criar schema Silver no Postgres

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="postgres",
    port=5432,
    dbname="moviepulse",
    user="moviepulse",
    password="moviepulse"
)
conn.autocommit = True
cur = conn.cursor()

ddl = """
-- Schema Silver
CREATE SCHEMA IF NOT EXISTS silver;

-- Catálogo de gêneros
CREATE TABLE IF NOT EXISTS silver.dim_genres (
    genre_id    INTEGER PRIMARY KEY,
    genre_name  VARCHAR(100) NOT NULL
);

-- Dimensão de filmes (deduplicada)
CREATE TABLE IF NOT EXISTS silver.dim_movies (
    movie_id            INTEGER PRIMARY KEY,
    title               VARCHAR(500),
    original_title      VARCHAR(500),
    original_language   VARCHAR(10),
    overview            TEXT,
    poster_path         VARCHAR(300),
    release_date        DATE,
    adult               BOOLEAN,
    processed_at        TIMESTAMP
);

-- Bridge filme ↔ gênero (N:N)
CREATE TABLE IF NOT EXISTS silver.bridge_movie_genres (
    movie_id    INTEGER NOT NULL,
    genre_id    INTEGER NOT NULL,
    PRIMARY KEY (movie_id, genre_id)
);

-- Fato: métricas diárias de trending
CREATE TABLE IF NOT EXISTS silver.fact_trending (
    movie_id        INTEGER  NOT NULL,
    source_date     DATE     NOT NULL,
    popularity      FLOAT,
    vote_average    FLOAT,
    vote_count      INTEGER,
    extraction_ts   TIMESTAMP,
    processed_at    TIMESTAMP,
    PRIMARY KEY (movie_id, source_date)
);
"""

cur.execute(ddl)
print(" Schema Silver criado no Postgres!")

cur.close()
conn.close()

## 💾 Passo 9 — Gravar no Postgres via JDBC

> Usamos `mode="append"` com `upsert` manual via Postgres `ON CONFLICT DO UPDATE` para garantir idempotência — rodar o job duas vezes não duplica dados.

In [ ]:
JDBC_URL = "jdbc:postgresql://postgres:5432/moviepulse"
JDBC_PROPS = {
    "user":     "moviepulse",
    "password": "moviepulse",
    "driver":   "org.postgresql.Driver",
}

def write_to_postgres(df, table: str, mode: str = "append"):
    """Grava um DataFrame Spark no Postgres via JDBC."""
    (
        df.write
        .jdbc(
            url=JDBC_URL,
            table=table,
            mode=mode,
            properties=JDBC_PROPS,
        )
    )
    print(f" {df.count()} registros gravados em {table}")

print("Função write_to_postgres pronta.")

In [ ]:
# ── 1. dim_genres ────────────────────────────────────────────────
# overwrite: catálogo de gêneros é pequeno e estável, substituir é seguro
write_to_postgres(df_genres, "silver.dim_genres", mode="overwrite")

In [ ]:
# ── 2. dim_movies ────────────────────────────────────────────────
# Upsert: atualiza filmes existentes, insere novos
# O Spark JDBC não faz upsert nativo → gravamos em tabela temporária e fazemos MERGE no Postgres

write_to_postgres(df_dim_movies, "silver.dim_movies_staging", mode="overwrite")

conn = psycopg2.connect(host="postgres", port=5432, dbname="moviepulse", user="moviepulse", password="moviepulse")
conn.autocommit = True
cur = conn.cursor()

cur.execute("""
    INSERT INTO silver.dim_movies
        SELECT * FROM silver.dim_movies_staging
    ON CONFLICT (movie_id) DO UPDATE SET
        title               = EXCLUDED.title,
        original_title      = EXCLUDED.original_title,
        original_language   = EXCLUDED.original_language,
        overview            = EXCLUDED.overview,
        poster_path         = EXCLUDED.poster_path,
        release_date        = EXCLUDED.release_date,
        adult               = EXCLUDED.adult,
        processed_at        = EXCLUDED.processed_at;

    DROP TABLE IF EXISTS silver.dim_movies_staging;
""")

cur.close()
conn.close()
print(" dim_movies: upsert concluído!")

In [ ]:
# ── 3. bridge_movie_genres ───────────────────────────────────────
write_to_postgres(df_bridge, "silver.bridge_movie_genres_staging", mode="overwrite")

conn = psycopg2.connect(host="postgres", port=5432, dbname="moviepulse", user="moviepulse", password="moviepulse")
conn.autocommit = True
cur = conn.cursor()

cur.execute("""
    INSERT INTO silver.bridge_movie_genres
        SELECT * FROM silver.bridge_movie_genres_staging
    ON CONFLICT (movie_id, genre_id) DO NOTHING;

    DROP TABLE IF EXISTS silver.bridge_movie_genres_staging;
""")

cur.close()
conn.close()
print bridge_movie_genres: upsert concluído!")

In [ ]:
# ── 4. fact_trending ─────────────────────────────────────────────
write_to_postgres(df_fact_trending, "silver.fact_trending_staging", mode="overwrite")

conn = psycopg2.connect(host="postgres", port=5432, dbname="moviepulse", user="moviepulse", password="moviepulse")
conn.autocommit = True
cur = conn.cursor()

cur.execute("""
    INSERT INTO silver.fact_trending
        SELECT * FROM silver.fact_trending_staging
    ON CONFLICT (movie_id, source_date) DO UPDATE SET
        popularity      = EXCLUDED.popularity,
        vote_average    = EXCLUDED.vote_average,
        vote_count      = EXCLUDED.vote_count,
        extraction_ts   = EXCLUDED.extraction_ts,
        processed_at    = EXCLUDED.processed_at;

    DROP TABLE IF EXISTS silver.fact_trending_staging;
""")

cur.close()
conn.close()
print("fact_trending: upsert concluído!")

## ✅ Passo 10 — Validação final

In [ ]:
conn = psycopg2.connect(host="postgres", port=5432, dbname="moviepulse", user="moviepulse", password="moviepulse")
cur = conn.cursor()

queries = {
    "dim_genres":          "SELECT COUNT(*) FROM silver.dim_genres",
    "dim_movies":          "SELECT COUNT(*) FROM silver.dim_movies",
    "bridge_movie_genres": "SELECT COUNT(*) FROM silver.bridge_movie_genres",
    "fact_trending":       "SELECT COUNT(*) FROM silver.fact_trending",
}

print("📊 Contagem de registros na Silver:\n")
for tabela, sql in queries.items():
    cur.execute(sql)
    count = cur.fetchone()[0]
    print(f"  silver.{tabela:<25} → {count:>6} registros")

cur.close()
conn.close()

print("\n Pipeline Bronze → Silver concluído com sucesso!")

In [ ]:
# Preview: top 10 filmes mais populares na Silver
conn = psycopg2.connect(host="postgres", port=5432, dbname="moviepulse", user="moviepulse", password="moviepulse")
cur = conn.cursor()

cur.execute("""
    SELECT
        m.title,
        t.popularity,
        t.vote_average,
        t.vote_count,
        t.source_date,
        STRING_AGG(g.genre_name, ', ' ORDER BY g.genre_name) AS genres
    FROM silver.fact_trending t
    JOIN silver.dim_movies          m ON m.movie_id = t.movie_id
    JOIN silver.bridge_movie_genres b ON b.movie_id = t.movie_id
    JOIN silver.dim_genres          g ON g.genre_id = b.genre_id
    WHERE t.source_date = (SELECT MAX(source_date) FROM silver.fact_trending)
    GROUP BY m.title, t.popularity, t.vote_average, t.vote_count, t.source_date
    ORDER BY t.popularity DESC
    LIMIT 10;
""")

rows = cur.fetchall()
cur.close()
conn.close()

print(f"{'Título':<40} {'Pop':>7} {'Nota':>5} {'Votos':>7}  Gêneros")
print("-" * 100)
for title, pop, avg, votes, date, genres in rows:
    print(f"{title:<40} {pop:>7.1f} {avg:>5.1f} {votes:>7}  {genres}")

---
## 📋 Resumo do que foi feito

| Tabela | Estratégia | Chave |  
|--------|-----------|-------|  
| `silver.dim_genres` | overwrite | `genre_id` |  
| `silver.dim_movies` | upsert | `movie_id` |  
| `silver.bridge_movie_genres` | insert ignore | `(movie_id, genre_id)` |  
| `silver.fact_trending` | upsert | `(movie_id, source_date)` |  

**Próximo passo:** criar a camada Gold com KPIs agregados para o Metabase.